# 目录
## 1. 导入模块
## 2. 从CSV文件里读取数据
### 2.1 获取文件名列表
### 2.2 从csv文件里读取数据，以dataset返回
  - `tf.data.Dataset.list_files`
  - `tf.data.Dataset.from_tensor_slices`
  - `interleave`
  - `tf.data.TextLineDataset` 和 `skip`
  - `tf.io.decode_csv`
  - `tf.stack`
  - `map`
  - `shuffle`
  - `repeat`
  - `batch`

## 3. 把数据写入tf_records
### 3.1 定义写入方法
  - `tf.io.TFRecordOptions`
  - `tf.io.TFRecordWriter`
  - `skip`
  - `tf.train.Example`
  - `tf.train.Features`
  - `tf.train.Feature`
  - `tf.train.FloatList`
  - `SerializeToString`
  
### 3.2 普通写入
### 3.3 ZIP写入

## 4. tfrecord 读取
### 4.1 读取方法的定义
  - `tf.data.Dataset.list_files`
  - `interleave`
  - `tf.data.TFRecordDataset`
  - `tf.io.FixedLenFeature`
  - `tf.io.parse_single_example`
  - `map`
  - `shuffle`
  - `repeat`
  - `batch`
  
### 4.2 测试
### 4.3 train_dataset、valid_dataset、test_dataset的获取
## 5. 模型建立、训练、测试

## 1. 导入模块

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn

from tensorflow import keras
import tensorflow as tf
import sys
import os
import time
import datetime

for module in [np, pd, mpl, sklearn, keras, tf]:
    print(module.__name__, module.__version__)

numpy 1.18.1
pandas 0.25.3
matplotlib 3.1.2
sklearn 0.22.1
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf
tensorflow 2.1.0


## 2. 从CSV文件里读取数据
>第二个文件笔记生成的加利福尼亚房价csv数据，并按照该文件笔记的方法生成dataset读取

### 2.1 获取文件名列表

In [2]:
def get_filenames_by_prefix(source_dir, prefix_name):
    all_file = os.listdir(source_dir)
    return [os.path.join(source_dir, file) for file in all_file if file.startswith(prefix_name) and file.endswith(".csv")]

train_filenames = get_filenames_by_prefix("generate_csv/", "train")
valid_filenames = get_filenames_by_prefix("generate_csv/", "valid")
test_filenames = get_filenames_by_prefix("generate_csv/", "test")

print(train_filenames)
print(valid_filenames)
print(test_filenames)

['generate_csv/train_11.csv', 'generate_csv/train_01.csv', 'generate_csv/train_19.csv', 'generate_csv/train_09.csv', 'generate_csv/train_05.csv', 'generate_csv/train_14.csv', 'generate_csv/train_15.csv', 'generate_csv/train_16.csv', 'generate_csv/train_10.csv', 'generate_csv/train_00.csv', 'generate_csv/train_17.csv', 'generate_csv/train_06.csv', 'generate_csv/train_03.csv', 'generate_csv/train_07.csv', 'generate_csv/train_12.csv', 'generate_csv/train_08.csv', 'generate_csv/train_18.csv', 'generate_csv/train_02.csv', 'generate_csv/train_04.csv', 'generate_csv/train_13.csv']
['generate_csv/valid_04.csv', 'generate_csv/valid_08.csv', 'generate_csv/valid_07.csv', 'generate_csv/valid_06.csv', 'generate_csv/valid_01.csv', 'generate_csv/valid_02.csv', 'generate_csv/valid_03.csv', 'generate_csv/valid_00.csv', 'generate_csv/valid_09.csv', 'generate_csv/valid_05.csv']
['generate_csv/test_05.csv', 'generate_csv/test_06.csv', 'generate_csv/test_08.csv', 'generate_csv/test_02.csv', 'generate_csv/t

### 2.2 从csv文件里读取数据，以dataset返回

In [3]:
def csv_reader_dataset(filenames, n_readers=5, n_fields=9, n_threads=5, epochs=None, batch_size=32, shuffle=True):

    # 1. dataset 分装文件列表
    filenames_dataset = tf.data.Dataset.from_tensor_slices(filenames)
    # filenames_dataset = tf.data.Dataset.list_files(filenames)
    
    # 2. interleave 根据文件名 读取文件数据
    raw_dataset = filenames_dataset.interleave( # interleave 下面函数返回的元素，组合成一个dataset
        lambda filename: tf.data.TextLineDataset(filename).skip(1),
        cycle_length=n_readers
    )
    
    # 3. 使用map 和 tf.io.decode_csv 对应解析
    def parse_csv_line(line, n_fields):
        default_fields = [tf.constant(np.nan)]* n_fields
        parsed_line = tf.io.decode_csv(line, default_fields) # 解析出来的是一个列表，列表的元素是一个0维的tensor

        # tf.stack 把 parsed_line 转化为一个向量tensor
        x = tf.stack(parsed_line[:-1])
        y = tf.stack(parsed_line[-1:])

        return x, y
    
    dataset = raw_dataset.map(  # map 不改变dataset的元素数量，只改变元素的数据结构
        map_func=lambda line: parse_csv_line(line, n_fields),
        num_parallel_calls=n_threads
    )
    
    # 4. shuffle repeat batch
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    
    return dataset

train_dataset = csv_reader_dataset(train_filenames, n_readers=5, n_fields=9, n_threads=5, epochs=1, batch_size=32, shuffle=False)
valid_dataset = csv_reader_dataset(valid_filenames, n_readers=5, n_fields=9, n_threads=5, epochs=1, batch_size=32, shuffle=False)
test_dataset = csv_reader_dataset(test_filenames, n_readers=5, n_fields=9, n_threads=5, epochs=1, batch_size=32, shuffle=False)

## 3. 把数据写入tf_records

### 3.1 定义写入方法

In [4]:
def csv_dataset_to_tfrecords(base_filenames, dataset, n_shares, steps_per_share, compression_type=None):
    
    all_filenames = []
    for share_id in range(n_shares):
        
        # 写入文件的路径
        filename_fullpath = "{}_{:05d}-of-{:05d}".format(base_filenames, share_id, n_shares)
        all_filenames.append(filename_fullpath)
        
        # 获取tf_record 写入对象
        options = tf.io.TFRecordOptions(compression_type=compression_type)
        with tf.io.TFRecordWriter(filename_fullpath, options=options) as writer:
            
            # 从dataset里遍历数据
            for batch_x, batch_y in dataset.skip(share_id*steps_per_share).take(steps_per_share):
                
                # 对一个batch的数据再次遍历
                for example_x, example_y in zip(batch_x, batch_y):
                    
                    # 对一个样本数据分装成example
                    example = tf.train.Example(
                        features=tf.train.Features(
                            feature={
                                "input_features": tf.train.Feature(float_list=tf.train.FloatList(value=example_x)),
                                "label": tf.train.Feature(float_list=tf.train.FloatList(value=example_y))
                            }
                        )
                    )
                    
                    # 序列化example
                    serialized_example = example.SerializeToString()
                    
                    # 执行写操作
                    writer.write(serialized_example)
    return all_filenames
                    


### 3.2 普通写入

In [5]:
n_shares = 20
batch_size = 32
train_steps_per_share = 11610 //batch_size//n_shares
valid_steps_per_share = 3880 //batch_size//n_shares
test_steps_per_share = 5170 //batch_size//n_shares


output_dir = "generate_tfrecords"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
train_basename = os.path.join(output_dir, "train")
valid_basename = os.path.join(output_dir, "valid")
test_basename = os.path.join(output_dir, "test")

train_tfrecords_filenames = csv_dataset_to_tfrecords(train_basename, train_dataset, n_shares, train_steps_per_share, compression_type=None)
valid_tfrecords_filenames = csv_dataset_to_tfrecords(valid_basename, valid_dataset, n_shares, valid_steps_per_share, compression_type=None)
test_tfrecords_filenames = csv_dataset_to_tfrecords(test_basename, test_dataset, n_shares, test_steps_per_share, compression_type=None)

print(train_tfrecords_filenames)
print(valid_tfrecords_filenames)
print(test_tfrecords_filenames)

['generate_tfrecords/train_00000-of-00020', 'generate_tfrecords/train_00001-of-00020', 'generate_tfrecords/train_00002-of-00020', 'generate_tfrecords/train_00003-of-00020', 'generate_tfrecords/train_00004-of-00020', 'generate_tfrecords/train_00005-of-00020', 'generate_tfrecords/train_00006-of-00020', 'generate_tfrecords/train_00007-of-00020', 'generate_tfrecords/train_00008-of-00020', 'generate_tfrecords/train_00009-of-00020', 'generate_tfrecords/train_00010-of-00020', 'generate_tfrecords/train_00011-of-00020', 'generate_tfrecords/train_00012-of-00020', 'generate_tfrecords/train_00013-of-00020', 'generate_tfrecords/train_00014-of-00020', 'generate_tfrecords/train_00015-of-00020', 'generate_tfrecords/train_00016-of-00020', 'generate_tfrecords/train_00017-of-00020', 'generate_tfrecords/train_00018-of-00020', 'generate_tfrecords/train_00019-of-00020']
['generate_tfrecords/valid_00000-of-00020', 'generate_tfrecords/valid_00001-of-00020', 'generate_tfrecords/valid_00002-of-00020', 'generate

### 3.3 ZIP写入 

In [6]:
zip_output_dir = "generate_tfrecords_zip"
if not os.path.exists(zip_output_dir):
    os.mkdir(zip_output_dir)
    
zip_train_basename = os.path.join(zip_output_dir, "train")
zip_valid_basename = os.path.join(zip_output_dir, "valid")
zip_test_basename = os.path.join(zip_output_dir, "test")

train_tfrecords_filenames_zip = csv_dataset_to_tfrecords(
    zip_train_basename, train_dataset, n_shares, train_steps_per_share, compression_type="GZIP")
valid_tfrecords_filenames_zip = csv_dataset_to_tfrecords(
    zip_valid_basename, valid_dataset, n_shares, valid_steps_per_share, compression_type="GZIP")
test_tfrecords_filenames_zip = csv_dataset_to_tfrecords(
    zip_test_basename, test_dataset, n_shares, test_steps_per_share, compression_type="GZIP")

print(train_tfrecords_filenames_zip)
print(valid_tfrecords_filenames_zip)
print(test_tfrecords_filenames_zip)

['generate_tfrecords_zip/train_00000-of-00020', 'generate_tfrecords_zip/train_00001-of-00020', 'generate_tfrecords_zip/train_00002-of-00020', 'generate_tfrecords_zip/train_00003-of-00020', 'generate_tfrecords_zip/train_00004-of-00020', 'generate_tfrecords_zip/train_00005-of-00020', 'generate_tfrecords_zip/train_00006-of-00020', 'generate_tfrecords_zip/train_00007-of-00020', 'generate_tfrecords_zip/train_00008-of-00020', 'generate_tfrecords_zip/train_00009-of-00020', 'generate_tfrecords_zip/train_00010-of-00020', 'generate_tfrecords_zip/train_00011-of-00020', 'generate_tfrecords_zip/train_00012-of-00020', 'generate_tfrecords_zip/train_00013-of-00020', 'generate_tfrecords_zip/train_00014-of-00020', 'generate_tfrecords_zip/train_00015-of-00020', 'generate_tfrecords_zip/train_00016-of-00020', 'generate_tfrecords_zip/train_00017-of-00020', 'generate_tfrecords_zip/train_00018-of-00020', 'generate_tfrecords_zip/train_00019-of-00020']
['generate_tfrecords_zip/valid_00000-of-00020', 'generate_t

## 4. tfrecord 读取
>套路跟`csv_reader_dataset`方法一样

### 4.1 读取方法的定义

In [7]:
def tfrecords_reader_dataset(
    filenames, n_readers=5,n_threads=5, epochs=None, batch_size=32, shuffle=True, compression_type=None):

    # 1. dataset 分装文件列表
    #filenames_dataset = tf.data.Dataset.from_tensor_slices(filenames)
    filenames_dataset = tf.data.Dataset.list_files(filenames)
    
    # 2. interleave 根据文件名 读取文件数据
    raw_dataset = filenames_dataset.interleave( # interleave 下面函数返回的元素，组合成一个dataset
        lambda filename: tf.data.TFRecordDataset(filename, compression_type=compression_type),
        cycle_length=n_readers
    )
    
    # 3. 使用map 和 tf.io.parse_single_example 对应解析
    def parse_serialized_example(serialized_example):
        expected_features = {
            "input_features": tf.io.FixedLenFeature([8], dtype=tf.float32),
            "label": tf.io.FixedLenFeature([1], dtype=tf.float32)
        }
        example = tf.io.parse_single_example(serialized_example, expected_features)
        return example["input_features"], example["label"]
        
    
    dataset = raw_dataset.map(  # map 不改变dataset的元素数量，只改变元素的数据结构
        map_func=parse_serialized_example,
        num_parallel_calls=n_threads
    )
    
    # 4. shuffle repeat batch
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    
    return dataset

### 4.2 测试

In [8]:
test_dataset= tfrecords_reader_dataset(
    test_tfrecords_filenames_zip, n_readers=5,n_threads=5, epochs=1, batch_size=2, shuffle=False, compression_type="GZIP")

for x, y in test_dataset.take(3):
    print(x)
    print(y)

tf.Tensor(
[[ 0.17960499  1.8773133  -0.19291258 -0.01163158 -0.7089215  -0.09591901
   1.0027343  -1.347747  ]
 [-0.39126503 -0.12413459 -0.4806664  -0.09861694  0.6785487   0.01911838
  -0.8130038   0.6002651 ]], shape=(2, 8), dtype=float32)
tf.Tensor(
[[2.75]
 [2.15]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[ 1.2281055   1.7171974   0.17694804  0.06742893 -0.6835753  -0.03524943
  -0.7334482   0.5902753 ]
 [ 0.47200948  0.4362708   0.24721687 -0.18787888 -0.10248999  0.00257246
  -0.8036443   0.7700918 ]], shape=(2, 8), dtype=float32)
tf.Tensor(
[[5.00001]
 [2.107  ]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[ 0.6331164   0.3562129   0.15536398 -0.10958562 -0.63851535  0.00521998
  -0.6398535   0.5503161 ]
 [-0.5909521   1.6371396  -0.81227165 -0.10447624 -0.00673764  0.13667765
  -0.75216717  0.68018353]], shape=(2, 8), dtype=float32)
tf.Tensor(
[[2.103]
 [1.46 ]], shape=(2, 1), dtype=float32)


### 4.3 train_dataset、valid_dataset、test_dataset的获取

In [13]:
train_dataset= tfrecords_reader_dataset(
    train_tfrecords_filenames_zip, n_readers=5,n_threads=5, epochs=None, batch_size=32, shuffle=True, compression_type="GZIP")

valid_dataset= tfrecords_reader_dataset(
    valid_tfrecords_filenames_zip, n_readers=5,n_threads=5, epochs=1, batch_size=32, shuffle=False, compression_type="GZIP")

test_dataset= tfrecords_reader_dataset(
    test_tfrecords_filenames_zip, n_readers=5,n_threads=5, epochs=1, batch_size=32, shuffle=False, compression_type="GZIP")

## 5. 模型建立、训练、测试

In [10]:
model = keras.models.Sequential([
    keras.layers.Dense(100, input_shape=[8], activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(loss="mean_squared_error", optimizer="sgd")

In [11]:
model.fit(
    train_dataset, 
    validation_data=valid_dataset, 
    steps_per_epoch=11160//32, 
    validation_steps=3870//32, 
    epochs=100
)

Train for 348 steps, validate for 120 steps
Epoch 1/100
348/348 [==============================] - 1s 3ms/step - loss: 0.7141 - val_loss: 0.7325
Epoch 2/100
348/348 [==============================] - 1s 2ms/step - loss: 0.8888 - val_loss: 0.4738
Epoch 3/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4548 - val_loss: 1.0081
Epoch 4/100
348/348 [==============================] - 1s 2ms/step - loss: 0.5834 - val_loss: 0.5594
Epoch 5/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4102 - val_loss: 0.4088
Epoch 6/100
348/348 [==============================] - 1s 2ms/step - loss: 1.1638 - val_loss: 0.4572
Epoch 7/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3980 - val_loss: 0.3726
Epoch 8/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3460 - val_loss: 0.3450
Epoch 9/100
348/348 [==============================] - 1s 3ms/step - loss: 0.3431 - val_loss: 0.3431
Epoch 10/100
348/348 [=========================

Epoch 81/100
348/348 [==============================] - 1s 2ms/step - loss: 0.2395 - val_loss: 0.2729
Epoch 82/100
348/348 [==============================] - 1s 2ms/step - loss: 0.2451 - val_loss: 0.2703
Epoch 83/100
348/348 [==============================] - 1s 2ms/step - loss: 0.2497 - val_loss: 0.2831
Epoch 84/100
348/348 [==============================] - 1s 2ms/step - loss: 0.2465 - val_loss: 0.2659
Epoch 85/100
348/348 [==============================] - 1s 2ms/step - loss: 0.2369 - val_loss: 0.2804
Epoch 86/100
348/348 [==============================] - 1s 2ms/step - loss: 0.2473 - val_loss: 0.2733
Epoch 87/100
348/348 [==============================] - 1s 2ms/step - loss: 0.2454 - val_loss: 0.2712
Epoch 88/100
348/348 [==============================] - 1s 2ms/step - loss: 0.2449 - val_loss: 0.2719
Epoch 89/100
348/348 [==============================] - 1s 2ms/step - loss: 0.2421 - val_loss: 0.2744
Epoch 90/100
348/348 [==============================] - 1s 2ms/step - loss: 0.2410

In [12]:
model.evaluate(test_dataset)

    160/Unknown - 0s 2ms/step - loss: 0.2834

0.2834040036890656